In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
from shapely.errors import GEOSException
import shapely.wkt

from shapely_test import shapes
from shapely_test.cases import TestCase

print(f"{shapely.__version__ = }")

In [ ]:
def plot_generator(case: TestCase):
    print(f"{case.gdf1_is_valid() = } and {case.gdf2_is_valid() = }")

    fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(40, 20), dpi=72)

    # intersection
    ax = axes[0, 0]
    ax.set_title("1 Intersection 2")
    case.gdf1.plot(ax=ax, alpha=0.5, color="magenta")
    case.gdf2.plot(ax=ax, alpha=0.5, color="cyan")
    try:
        case.gdf1_gdf2_overlay_intersection().plot(ax=ax, alpha=0.5, color="black")
    except GEOSException as e:
        plot_error("gdf1_gdf2_overlay_intersection", e, ax)

    # union
    ax = axes[1, 0]
    ax.set_title("1 Union 2")
    case.gdf1.plot(ax=ax, alpha=0.5, color="magenta")
    case.gdf2.plot(ax=ax, alpha=0.5, color="cyan")
    try:
        case.gdf1_gdf2_overlay_union().plot(ax=ax, alpha=0.4, color="black")
    except GEOSException as e:
        plot_error("gdf1_gdf2_overlay_union", e, ax)

    # diff 1 -> 2
    ax = axes[0, 1]
    ax.set_title("1 Difference 2")
    case.gdf1.plot(ax=ax, alpha=0.5, color="magenta")
    case.gdf2.plot(ax=ax, alpha=0.5, color="cyan")
    try:
        case.gdf1_gdf2_overlay_difference().plot(ax=ax, alpha=0.5, color="black")
    except GEOSException as e:
        plot_error("gdf1_gdf2_overlay_difference", e, ax)

    # diff 2 -> 1
    ax = axes[1, 1]
    ax.set_title("2 Difference 1")
    case.gdf1.plot(ax=ax, alpha=0.5, color="magenta")
    case.gdf2.plot(ax=ax, alpha=0.5, color="cyan")
    try:
        case.gdf2_gdf1_overlay_difference().plot(ax=ax, alpha=0.5, color="black")
    except GEOSException as e:
        plot_error("gdf2_gdf1_overlay_difference", e, ax)

    # diff 1 -> 1
    ax = axes[0, 2]
    ax.set_title("1 Difference 1")
    case.gdf1.plot(ax=ax, alpha=0.5, color="magenta")
    try:
        gdf = case.gdf1_gdf1_overlay_difference()
        if len(gdf) > 0:
            plot_gdf_and_zoom(gdf, ax)
    except GEOSException as e:
        plot_error("gdf1_gdf1_overlay_difference", e, ax)

    # union 1 -> 1
    ax = axes[1, 2]
    ax.set_title("1 Union 1")
    case.gdf1.plot(ax=ax, alpha=0.5, color="magenta")
    try:
        case.gdf1_gdf1_overlay_union().plot(ax=ax, alpha=0.5, color="black")
    except GEOSException as e:
        plot_error("gdf1_gdf1_overlay_union", e, ax)

    # diff 2 -> 2
    ax = axes[0, 3]
    ax.set_title("2 Difference 2")
    case.gdf2.plot(ax=ax, alpha=0.5, color="cyan")
    try:
        gdf = case.gdf2_gdf2_overlay_difference()
        if len(gdf) > 0:
            plot_gdf_and_zoom(gdf, ax)
    except GEOSException as e:
        plot_error("gdf2_gdf2_overlay_difference", e, ax)

    # union 2 -> 2
    ax = axes[1, 3]
    ax.set_title("2 Union 2")
    case.gdf2.plot(ax=ax, alpha=0.5, color="cyan")
    try:
        case.gdf2_gdf2_overlay_union().plot(ax=ax, alpha=0.5, color="black")
    except GEOSException as e:
        plot_error("gdf2_gdf2_overlay_union", e, ax)

    for ax in axes.flatten():
        ax.axis("off")
    fig.tight_layout()

    plt.show()
    plt.close(fig)


def plot_error(name: str, e: GEOSException, ax: plt.Axes):
    start = "TopologyException: found non-noded intersection between "
    e = str(e)
    if e.startswith(start):
        ab, cd = e[len(start):].split(" and ")
        cd = cd.split(" at ")[0]
        ab, cd = shapely.wkt.loads(ab), shapely.wkt.loads(cd)
        plot_gdf_and_zoom(gpd.GeoDataFrame([ab, cd], columns=["geometry"]), ax)
    else:
        print(e)
        print(start)


def plot_gdf_and_zoom(gdf: gpd.GeoDataFrame, ax: plt.Axes):
    gdf.plot(ax=ax, facecolor="none", edgecolor="black")
    xmin, ymin, xmax, ymax = gdf.unary_union.bounds
    delta = max(xmax - xmin, ymax - ymin) // 2 + 1
    xcenter, ycenter = 0.5 * (xmin + xmax), 0.5 * (ymin + ymax)
    ax.set_xlim([xcenter - delta, xcenter + delta])
    ax.set_ylim([ycenter - delta, ycenter + delta])

In [ ]:
plot_generator(case=TestCase(shapes.RandomCenterTargetsGenerator(), seed=207))

In [ ]:
plot_generator(case=TestCase(shapes.RandomCenterTargetsGenerator(), seed=254))

In [ ]:
plot_generator(case=TestCase(shapes.RandomRadiusTargetsGenerator(), seed=324))

In [ ]:
case = TestCase(shapes.RandomSpotsGenerator(), seed=323)
plot_generator(case)

In [ ]:
plot_generator(case=TestCase(shapes.RandomSpotsGenerator(), seed=439))